# GMRES

In [1]:
import numpy as np
from numpy.linalg import qr

from functions import arnoldi, back_substitution

from scipy.sparse.linalg import lsqr
from scipy.linalg import solve_triangular

For some reason I cannot understand, the following code did not work with 

y = solve_triangular(R[:-1, :] , beta*Q[0][:-1], lower = True).

Tried other solvers like scipy.sparse.linalg.lsqr and it worked. So I decided to implement back substitution manually.

In [2]:
def GMRES(A, b, x0, k_max = None , epsilon = 1e-8):
    """
    Generalized Minimal RESidual method for solving linear systems.
    
    Parameters:
    -----------
    A : numpy.ndarray
        Coefficient matrix of the linear system.
        
    b : numpy.ndarray
        Right-hand side vector of the linear system.
        
    x0 : numpy.ndarray
        Initial guess for the solution.
        
    k_max : int
        Maximum number of iterations.
        
    epsilon : float, optional
        Tolerance for convergence.
    
    Output:
    --------
    numpy.ndarray
        Approximate solution to the linear system.
    """
    
    n = A.shape[0]
    
    if (k_max is None):
        k_max = n
        
    elif k_max > n:
        k_max = n
        
    r0 = b - A @ x0
    p0 = np.linalg.norm(r0)
    beta = p0
    pk = p0
    k = 0
    
    while pk > epsilon*p0 and k < k_max:
        
        k += 1
        
        V, H = arnoldi(A, b, r0, k) # Arnoldi algorithm to generate V_{k+1} and H_{k+1, k}
        
        Q, R = qr(H, mode = 'complete')
        
        pk = abs(beta*Q[0, k]) # Compute norm of residual vector
        
        yk = back_substitution(R[:-1, :] , beta*Q[0][:-1])
    
        xk = x0 + V[:, :-1]@yk # Compute the new approximation x0 + V_{k}y
        
    return xk, pk

# Unit tests

#### Simple systems. The results are the correct ones.

In [5]:
A = np.array([[1, 3, 1], [3, 2, 4],  [1, 2, 0]])
b = np.array([3, 2, 1])
x0 = np.array([1, 2, 0])

x = GMRES(A, b, x0) # Not specifying max_iter it takes n = 3 iterations
x

Converged in 3 iterations.


(array([-1.25 ,  1.125,  0.875]), 4.866664144819235e-16)

#### Comparison with Scipy GMRES implementation

In [6]:
import scipy.sparse.linalg as spla

A = np.array([[1, 1, 4, 9], [3, 4, 6, 9], [4, 1, 1, 3], [3, 2, 1, 1]])
b = np.array([3, 2, 2, -3])
x0 = np.array([0, 0, 1, 0])
spla.gmres(A,b,x0, restart = None)[0], GMRES(A, b, x0, 4) # Converges in n = 4 iterations! God

Converged in 4 iterations.


(array([ 2.96296296, -9.44444444, 10.7037037 , -3.7037037 ]),
 (array([ 2.96296296, -9.44444444, 10.7037037 , -3.7037037 ]),
  1.0997010364081157e-14))